<a href="https://colab.research.google.com/github/fabian-fuentes/DSGE/blob/main/NonRicardianAgents/Taxes_chapter7_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taxes

In [1]:
# install packages
install.packages('nleqslv')
install.packages('Rcpp')
install.packages('reshape2')
install.packages("https://gecon.r-forge.r-project.org/files/gEcon_1.2.3.tar.gz", repos = NULL, type = "source")

In [2]:
library(nleqslv)
library(Rcpp)
library(gEcon)

Loading required package: MASS

Loading required package: Matrix

This is gEcon version 1.2.3 (2025-04-13)
http://gecon.r-forge.r-project.org/



In [3]:
download.file("https://raw.githubusercontent.com/fabian-fuentes/DSGE/main/NonRicardianAgents/Taxes_chapter7.gcn",
              destfile = "Taxes_chapter7.gcn", mode = "wb")

In [4]:
 gcn.file<-"Taxes_chapter7.gcn"

#create a model from gcc.file
  model <- make_model(gcn.file)

model parsed in 0.01s
model loaded in 0.06s


In [5]:
#set initial conditions
  model <- initval_var(model, list(C = 0.8,
                                    L_s = 0.3,
                                    K_s = 3.5,
                                    I = 0.2,
                                    Y = 1.0,
                                    A = 1.0,
                                    W=(1-0.350)*(1.0/0.3),
                                    r=(0.350)*(1.0/3.5),
                                    pi=0.0,
                                    PI=0.0 ,
                                    U = 1.0,
                                    G_inc = 0.116*0.8 + 0.348*(1-0.350)*(1.0/0.3)*0.3 + 0.225*((0.350)*(1.0/3.5)-0.06)*3.5  ,
                                    GT = 0.116*0.8 + 0.348*(1-0.350)*(1.0/0.3)*0.3 + 0.225*((0.350)*(1.0/3.5)-0.06)*3.5,
                                    lambda_c = 1.0,
                                    T_c = 0.116*0.8,
                                    T_l = 0.348*(1-0.350)*(1.0/0.3)*0.3,
                                    T_k = 0.225*((0.350)*(1.0/3.5)-0.06)*3.5
                                    )
                        )

#define steady state
 model <- steady_state(model)

Steady state has been FOUND


In [6]:
#=================================================================================================================
#create Laffer Curves
#=================================================================================================================

#create experimental design
  Xs <- expand.grid(list(tau_l=seq(0.0,1.0,0.1),tau_c=seq(0.0,1.0,0.1),tau_k=seq(0.0,1.0,0.1)))
  Xs$Run_ID <- 1:nrow(Xs)
  dim(Xs)

[1] 1331    4

In [8]:
laffer <- list()

for (i in 1:length(Xs$Run_ID)) {
  suppressWarnings({
    # i <- 67
    model_i <- initval_var(model, list(
      C = 0.8,
      L_s = 0.3,
      K_s = 3.5,
      I = 0.2,
      Y = 1.0,
      A = 1.0,
      W = (1 - 0.350) * (1.0 / 0.3),
      r = (0.350) * (1.0 / 3.5),
      pi = 0.0,
      PI = 0.0,
      U = 1.0,
      G_inc = Xs$tau_c[i]*0.8 + Xs$tau_l[i]*(1-0.350)*(1.0/0.3)*0.3 + Xs$tau_k[i]*((0.350)*(1.0/3.5)-0.06)*3.5,
      GT = Xs$tau_c[i]*0.8 + Xs$tau_l[i]*(1-0.350)*(1.0/0.3)*0.3 + Xs$tau_k[i]*((0.350)*(1.0/3.5)-0.06)*3.5,
      lambda_c = 0.1,
      T_c = Xs$tau_c[i]*0.8,
      T_l = Xs$tau_l[i]*(1-0.350)*(1.0/0.3)*0.3,
      T_k = Xs$tau_k[i]*((0.350)*(1.0/3.5)-0.06)*3.5
    ))

    model_i <- set_free_par(model_i, list(
      tau_l = Xs$tau_l[i],
      tau_c = Xs$tau_c[i],
      tau_k = Xs$tau_k[i]
    ), warnings = FALSE)

    model_i <- steady_state(model_i, last_solver_iter = TRUE)
    ss_i <- get_ss_values(model_i, silent = TRUE)
    ss_i <- data.frame(t(ss_i))
    ss_i$Run_ID <- Xs$Run_ID[i]
    laffer <- append(laffer, list(ss_i))
    rm(model_i)
  })
}


Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has been FOUND
Steady state has bee

In [9]:
#row bind results
 laffer <- do.call("rbind",laffer)

#merge Xs
 dim(laffer)
 laffer<-merge(laffer,Xs,by="Run_ID")
 dim(laffer)

#Identify cases of interests for laffer curves
laffer$labor_tax <- ifelse(laffer$tau_c==0 & laffer$tau_k==0,1,0)
laffer$capital_tax <- ifelse(laffer$tau_l==0 & laffer$tau_c==0,1,0)
laffer$consumption_tax <- ifelse(laffer$tau_l==0 & laffer$tau_k==0,1,0)

[1] 1331   18

[1] 1331   21

In [10]:
#write laffer curve database
write.csv(laffer,"laffer_curve.csv",row.names=FALSE)

In [11]:
#=================================================================================================================
#Introduce Productivity Shock
#=================================================================================================================
#estimate IRFs
  model <- set_shock_distr_par(model = model, distr_par = list("sd(epsilon_A)" = 0.01))
  #model<- compute_model_stats(model)
  model <- solve_pert(model)
#Compute Impulse Response Function (IRFs)
  model_irf <- compute_irf(model = model, variables = c('C','L_s','K_s','I','Y','A', 'G_inc','T_c','T_l','T_k'), sim_length = 40)


Warning message:
“the following variables will not be log-linearised as their steady-state values are equal or close to zero: "pi", "PI"”


Model has been SOLVED


In [12]:
#Extract simulation values
  library(reshape2)
  out <- as.data.frame.table(model_irf@sim)
  out<-dcast(out,Var2  ~ Var1 , value.var="Freq")
#print simulation
  write.csv(out, "out.csv",row.names=FALSE)

In [13]:
#Compute random path
  model_rp <- random_path(model = model, variables = c('C','L_s','K_s','I','Y','A', 'G_inc','T_c','T_l','T_k'), sim_length = 40)

#extract values of random path simulation
  out <- as.data.frame.table(model_rp@sim)
  out<-dcast(out,Var2  ~ Var1 , value.var="Freq")
  out$Case<-paste0("sd_epsilon_A = ",0.01)
  write.csv(out, "out_random_path.csv",row.names=FALSE)

# Introduce Multiple Shocks

In [14]:
download.file("https://raw.githubusercontent.com/fabian-fuentes/DSGE/main/NonRicardianAgents/nRa_chapter4.gcn",
              destfile = "nRa_chapter4.gcn", mode = "wb")

In [15]:
#now lets run different TFP shocks
sdepsilon_Avector<-seq(0.5,1.5, by= 0.1)
gcn.file<-"nRa_chapter4.gcn"

In [16]:
out_all <- list()

for (i in 1:length(sdepsilon_Avector))
{
#i<-1
#create a model from gcc.file
  model <- make_model(gcn.file)

#set parameter values

#set initial conditions
  model <- initval_var(model, list(C = 0.8,
                                    C_1=0.8,
                                    C_2 = 0.2,
                                    L_s = 0.3,
                                    L_1_s = 0.3,
                                    L_2_s = 0.3,
                                    K_s = 3.5,
                                    K_1_s = 4.0,
                                    I = 0.2,
                                    I_1 = 0.3,
                                    Y = 1.0,
                                    A = 1.0,
                                    W=(1-0.350)*(1.0/0.3),
                                    r=(0.350)*(1.0/3.5),
                                    pi=1.0,
                                    PI=1.0 ,
                                    U_1 = 1.0,
                                    U_2 = 2.0 ))

#define steady state
 model <- steady_state(model)

#Solving for dynamics
 model <- solve_pert(model)

#Introducing shocks
  model <- set_shock_distr_par(model = model, distr_par = list("sd(epsilon_A)" = sdepsilon_Avector[i]*0.01))

#Compute Impulse Response Function (IRFs)
  model_irf <- compute_irf(model = model, variables = c('C','C_1','C_2','L_s','L_1_s','L_2_s','K_s','K_1_s','I','I_1','Y','A'), sim_length = 40)

#Extract simulation values
  out <- as.data.frame.table(model_irf@sim)
  out<-dcast(out,Var2  ~ Var1 , value.var="Freq")
  out$Case<-paste0("sd_epsilon_A = ",sdepsilon_Avector[i])
  out$ShockType <- "A"
#save ouput
 out_all <- append(out_all,list(out))
}

model parsed in 0.01s
model loaded in 0.01s
Steady state has been FOUND


Warning message:
“the following variables will not be log-linearised as their steady-state values are equal or close to zero: "pi", "PI"”


Model has been SOLVED
model parsed in 0.01s
model loaded in 0.01s
Steady state has been FOUND


Warning message:
“the following variables will not be log-linearised as their steady-state values are equal or close to zero: "pi", "PI"”


Model has been SOLVED
model parsed in 0.01s
model loaded in 0.01s
Steady state has been FOUND


...

Model has been SOLVED
model parsed in 0.01s
model loaded in 0.01s
Steady state has been FOUND


Warning message:
“the following variables will not be log-linearised as their steady-state values are equal or close to zero: "pi", "PI"”


Model has been SOLVED
model parsed in 0.01s
model loaded in 0.01s
Steady state has been FOUND


Warning message:
“the following variables will not be log-linearised as their steady-state values are equal or close to zero: "pi", "PI"”


Model has been SOLVED


In [17]:
out_all <- do.call("rbind",out_all)
head(out_all)

,Var2,C,C_1,C_2,L_s,L_1_s,L_2_s,K_s,K_1_s,I,I_1,Y,A,Case,ShockType
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,1,0.003290261,0.002358608,0.004344904,0.0018717019,0.004205552,0,0.000957662,0.000957662,0.015961033,0.015961033,0.006216606,0.005000000,sd_epsilon_A = 0.5,A
2,2,0.003615648,0.002814885,0.004522122,0.0016087425,0.003614705,0,0.001770579,0.001770579,0.014506276,0.014506276,0.006130864,0.004750000,sd_epsilon_A = 0.5,A
3,3,0.003875214,0.003190270,0.004650581,0.0013760623,0.003091893,0,0.002455785,0.002455785,0.013190679,0.013190679,0.006026643,0.004512500,sd_epsilon_A = 0.5,A
4,4,0.004077264,0.003494682,0.004736755,0.0011704146,0.002629821,0,0.003028472,0.003028472,0.012000565,0.012000565,0.005907169,0.004286875,sd_epsilon_A = 0.5,A
5,5,0.004229177,0.003736950,0.004786385,0.0009888904,0.002221951,0,0.003502181,0.003502181,0.010923622,0.010923622,0.005775275,0.004072531,sd_epsilon_A = 0.5,A
6,6,0.004337510,0.003924927,0.004804559,0.0008288835,0.001862430,0,0.003888976,0.003888976,0.009948764,0.009948764,0.005633442,0.003868905,sd_epsilon_A = 0.5,A


In [18]:
download.file("https://raw.githubusercontent.com/fabian-fuentes/DSGE/main/NonRicardianAgents/nRa_chapter4_B_shock.gcn",
              destfile = "nRa_chapter4_B_shock.gcn", mode = "wb")

In [19]:
#now lets run an intertemporal substitution shock
gcn.file<-"nRa_chapter4_B_shock.gcn"
sdepsilon_Bvector<-seq(0.5,1.5, by= 0.1)

In [20]:
out_allB <- list()

for (i in 1:length(sdepsilon_Bvector))
{
#i<-1
 model <- make_model(gcn.file)

#set initial conditions
  model <- initval_var(model, list(C = 0.8,
                                    C_1=0.8,
                                    C_2 = 0.2,
                                    L_s = 0.3,
                                    L_1_s = 0.3,
                                    L_2_s = 0.3,
                                    K_s = 3.5,
                                    K_1_s = 4.0,
                                    I = 0.2,
                                    I_1 = 0.3,
                                    Y = 1.0,
                                    A = 1.0,
                                    W=(1-0.350)*(1.0/0.3),
                                    r=(0.350)*(1.0/3.5),
                                    pi=1.0,
                                    PI=1.0 ,
                                    U_1 = 1.0,
                                    U_2 = 2.0,
                                    B= 1))

#define steady state
 model <- steady_state(model)

#Solving for dynamics
 model <- solve_pert(model)

#Introducing shocks
  model <- set_shock_distr_par(model = model, distr_par = list("sd(epsilon_B)" = sdepsilon_Bvector[i]*0.01))

#Compute Impulse Response Function (IRFs)
  model_irf <- compute_irf(model = model, variables = c('C','C_1','C_2','L_s','L_1_s','L_2_s','K_s','K_1_s','I','I_1','Y','A'), sim_length = 40)

#Extract simulation values
  out <- as.data.frame.table(model_irf@sim)
  out<-dcast(out,Var2  ~ Var1 , value.var="Freq")
  out$Case<-paste0("sd_epsilon_B = ",sdepsilon_Bvector[i])
  out$ShockType <- "B"
#save ouput
  out_allB <- append(out_allB,list(out))
}

model parsed in 0.01s
model loaded in 0.01s
Steady state has been FOUND


Warning message:
“the following variables will not be log-linearised as their steady-state values are equal or close to zero: "pi", "PI"”


Model has been SOLVED
model parsed in 0.01s
model loaded in 0.02s
Steady state has been FOUND


Warning message:
“the following variables will not be log-linearised as their steady-state values are equal or close to zero: "pi", "PI"”


Model has been SOLVED
model parsed in 0.01s
model loaded in 0.01s
Steady state has been FOUND


...

Model has been SOLVED
model parsed in 0.01s
model loaded in 0.01s
Steady state has been FOUND


Warning message:
“the following variables will not be log-linearised as their steady-state values are equal or close to zero: "pi", "PI"”


Model has been SOLVED
model parsed in 0.01s
model loaded in 0.01s
Steady state has been FOUND


Warning message:
“the following variables will not be log-linearised as their steady-state values are equal or close to zero: "pi", "PI"”


Model has been SOLVED


In [21]:
out_allB <- do.call("rbind",out_allB)
head(out_allB)

,Var2,C,C_1,C_2,L_s,L_1_s,L_2_s,K_s,K_1_s,I,I_1,Y,A,Case,ShockType
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,1,-0.0182340466,-0.028169949,-0.0069864734,0.01996135,0.04485143,0,0.007013843,0.007013843,0.11689738,0.11689738,0.01297488,0,sd_epsilon_B = 0.5,B
2,2,-0.0137343570,-0.022556005,-0.0037481351,0.01772280,0.03982160,0,0.012967584,0.012967584,0.10624285,0.10624285,0.01397467,0,sd_epsilon_B = 0.5,B
3,3,-0.0098225564,-0.017648115,-0.0009639211,0.01572164,0.03532518,0,0.017985980,0.017985980,0.09660752,0.09660752,0.01475772,0,sd_epsilon_B = 0.5,B
4,4,-0.0064325446,-0.013367953,0.0014184299,0.01393332,0.03130697,0,0.022180294,0.022180294,0.08789122,0.08789122,0.01535175,0,sd_epsilon_B = 0.5,B
5,5,-0.0035052389,-0.009645464,0.0034455769,0.01233579,0.02771745,0,0.025649703,0.025649703,0.08000377,0.08000377,0.01578137,0,sd_epsilon_B = 0.5,B
6,6,-0.0009878481,-0.006418003,0.0051591589,0.01090925,0.02451214,0,0.028482559,0.028482559,0.07286398,0.07286398,0.01606841,0,sd_epsilon_B = 0.5,B


In [22]:
#print simulation
 out_all<-rbind(out_all,out_allB)
 write.csv(out_all,"out_sentivity_TFP.csv",row.names=FALSE)

In [23]:
#simulate the behavior of the economy using random path for shocks
#create a model from gcc.file
  gcn.file<-"nRa_chapter4.gcn"
  model <- make_model(gcn.file)

model parsed in 0.01s
model loaded in 0.01s


In [24]:
#set initial conditions
  model <- initval_var(model, list(C = 0.8,
                                    C_1=0.8,
                                    C_2 = 0.2,
                                    L_s = 0.3,
                                    L_1_s = 0.3,
                                    L_2_s = 0.3,
                                    K_s = 3.5,
                                    K_1_s = 4.0,
                                    I = 0.2,
                                    I_1 = 0.3,
                                    Y = 1.0,
                                    A = 1.0,
                                    W=(1-0.350)*(1.0/0.3),
                                    r=(0.350)*(1.0/3.5),
                                    pi=1.0,
                                    PI=1.0 ,
                                    U_1 = 1.0,
                                    U_2 = 2.0 ))

#define steady state
 model <- steady_state(model)

#Solving for dynamics
 model <- solve_pert(model)


Steady state has been FOUND


Warning message:
“the following variables will not be log-linearised as their steady-state values are equal or close to zero: "pi", "PI"”


Model has been SOLVED


In [25]:
#Introducing shocks
  model <- set_shock_distr_par(model = model, distr_par = list("sd(epsilon_A)" = 0.01))

#Compute random path
  model_rp <- random_path(model = model, variables = c('C','C_1','C_2','L_s','L_1_s','L_2_s','K_s','K_1_s','I','I_1','Y','A'), sim_length = 40)

#extract values of random path simulation
  out <- as.data.frame.table(model_rp@sim)
  out<-dcast(out,Var2  ~ Var1 , value.var="Freq")
  out$Case<-paste0("sd_epsilon_A = ",0.01)
  write.csv(out,"out_random_path.csv",row.names=FALSE)